# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

Import the BeautifulSoup Package

In [2]:
from bs4 import BeautifulSoup
import requests

Use BeautifulSoup to obtain the Table on the Wiki page. The latest table seems a little different than the one in the assignment, so used a older version of the table

In [3]:
URL = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=929562264" #older URL with table formatted similar to assignment
r = requests.get(URL)

soup = BeautifulSoup(r.content, "lxml")
Toronto_table = soup.find('table', {'class':'wikitable sortable'})

In [4]:
import pandas as pd

In [5]:
Toronto_df = pd.read_html(str(Toronto_table)) #Read the html table into a list of pandas dataframe
Toronto_df2 = Toronto_df[0] #Convert that list into a pandas dataframe

#Drop Boroughs that are 'Not assigned'. Assumption is that there are no typos or other formats of unassigned Boroughs
Toronto_df2.drop(Toronto_df2[Toronto_df2['Borough'] == 'Not assigned'].index, inplace = True)
Toronto_df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [6]:
#Next, replace all the not assigned Neighborhoods with the names of their Boroughs. Again assumption is that there are no typos or other formats of unassigned Neighbourhoods
Toronto_df2.Neighbourhood.replace('Not assigned', Toronto_df2.Borough, inplace = True)

Combine Neighborhoods in the Same Borough

In [7]:
#Use the groupby function to combine over postcose and Borough. The Apply feature allows us to join the Neighborhoods using a comma
Toronto_df2 = Toronto_df2.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
Toronto_df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
Toronto_df2.shape

(103, 3)

In [9]:
Toronto_coor = pd.read_csv('http://cocl.us/Geospatial_data')
Toronto_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
Toronto_df3 = Toronto_df2.join(Toronto_coor.set_index('Postal Code'), on = 'Postcode')

In [11]:
Toronto_df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

### 1. Toronto Map with Locations of Various Neighborhoods

In [135]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [136]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df3['Latitude'], Toronto_df3['Longitude'], Toronto_df3['Borough'], Toronto_df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 2. Explore The Downtown Toronto Borough

In [137]:
#Create DF with Downtown Toronto Neighborhoods and Coordinates
Downtown_Toronto_data = Toronto_df3[Toronto_df3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [138]:
#Find the coordinates of Downtown Toronto Borough
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [139]:
# create map of Downtown Toronto using latitude and longitude values
map_Downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add Neighborhood markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown)  
    
map_Downtown

### 2. Use Foursquare API TO Explore Venues in Downtown Toronto

In [140]:
CLIENT_ID = 'ZGXPQRMDRXAONZOFTROMVILSKSZF5EOWOQHUENJTN5POTNH1' # your Foursquare ID
CLIENT_SECRET = 'WBGUW21YJZVQDBB5LYO2L0QYPTJ1YRJYTYTW22HLA0GXY0PG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [141]:
#Generate Fourquare URL and get requests
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
results = requests.get(url).json()

In [142]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [143]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [144]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [145]:
Downtown_venues = getNearbyVenues(names=Downtown_Toronto_data['Neighbourhood'],
                                   latitudes=Downtown_Toronto_data['Latitude'],
                                   longitudes=Downtown_Toronto_data['Longitude']
                                  )
print(Downtown_venues.shape)
Downtown_venues.head()

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park
(1274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [146]:
Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",87,87,87,87,87,87
Christie,18,18,18,18,18,18
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [147]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 209 uniques categories.


In [148]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[144]] + list(Downtown_onehot.columns[:144])
Downtown_onehot = Downtown_onehot[fixed_columns]
#fixed_columns
Downtown_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [149]:
Downtown_onehot.shape

(1274, 145)

In [150]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.0,0.0,0.010000,0.010000,0.0,0.02,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.020000,0.0000,0.01,0.020000,0.0,0.000000,0.01,0.010000,0.01,0.000000,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.02,0.000000,0.070000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.020000,0.000000,0.0,0.010000,0.010000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.01,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.020000,0.0,0.000000,0.01,0.0,0.010000,0.01,0.000000,0.010000,0.000000,0.02,0.010000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.0,0.01,0.01,0.000000,0.020000,0.000000,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.01,0.0,0.000000,0.0
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,0.017857,0.000000,0.0,0.00,0.0,0.017857,0.0,0.017857,0.035714,0.000000,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.000000,0.0,0.017857,0.0000,0.000000,0.0000,0.00,0.017857,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.017857,0.035714,0.0,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.00,0.035714,0.089286,0.0,0.0,0.0,0.00,0.017857,0.000000,0.017857,0.017857,0.017857,0.0,0.000000,0.017857,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.0,0.0,0.000000,0.035714,0.00,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.017857,0.017857,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.00,0.017857,0.017857,0.000000,0.00,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.000000,0.000000,0.017857,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.017857,0.017857,0.000000,0.0,0.

The categorization in the dataframe above is a little messy. For example, coffee shops and cafes have two separate columns. Similarly, beer store, beer shops, and liquor stores are all different. 

For now, we will continue with the analysis without cleaning this up.

In [151]:
Downtown_grouped.shape

(18, 145)

## Analyze the Venues in the Different Neighborhoods

In [152]:
#Generate the top 5 venues in each Postal Code
num_top_venues = 5

for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.04
2             Bar  0.03
3           Hotel  0.02
4  Breakfast Spot  0.02


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1      Beer Bar  0.04
2        Bakery  0.04
3   Cheese Shop  0.04
4  Cocktail Bar  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3         Coffee Shop  0.06
4  Airport Food Court  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.04
2                Café  0.04
3              Bakery  0.04
4  Chinese Restaurant  0.02


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.05
2         Burger Joint  0.04
3       Ice Cream Shop  0.04
4  J

In [153]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [154]:
#We will look at the 10 most common venues in each postal code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Cosmetics Shop,Bookstore,Bakery,Hotel,Asian Restaurant,Gym,Lounge
1,Berczy Park,Coffee Shop,Beer Bar,Café,Bakery,Farmers Market,Cheese Shop,Cocktail Bar,Hotel,Fish Market,Basketball Stadium
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Coffee Shop,Boutique
3,"Cabbagetown, St. James Town",Coffee Shop,Italian Restaurant,Bakery,Café,Grocery Store,Breakfast Spot,Diner,Gastropub,Butcher,Gourmet Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Department Store,Café,Chinese Restaurant,Gym / Fitness Center


## Use k-means to cluster Neighborhoods

In [155]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)
Downtown_grouped_clustering.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue
0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.0,0.0,0.010000,0.010000,0.0,0.02,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.020000,0.0000,0.01,0.020000,0.0,0.000000,0.01,0.010000,0.01,0.000000,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.02,0.000000,0.070000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.020000,0.000000,0.0,0.010000,0.010000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.01,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.020000,0.0,0.000000,0.01,0.0,0.010000,0.01,0.000000,0.010000,0.000000,0.02,0.010000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.0,0.01,0.01,0.000000,0.020000,0.000000,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.01,0.0,0.000000,0.0
1,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.0,0.017857,0.000000,0.0,0.00,0.0,0.017857,0.0,0.017857,0.035714,0.000000,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.000000,0.0,0.017857,0.0000,0.000000,0.0000,0.00,0.017857,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.017857,0.035714,0.0,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.00,0.035714,0.089286,0.0,0.0,0.0,0.00,0.017857,0.000000,0.017857,0.017857,0.017857,0.0,0.000000,0.017857,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.0,0.0,0.000000,0.035714,0.00,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.017857,0.017857,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.00,0.017857,0.017857,0.000000,0.00,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.000000,0.000000,0.017857,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.017857,0.017857,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0

In [156]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18] 

array([4, 0, 3, 4, 0, 4, 2, 4, 0, 0, 0, 1, 0, 0, 4, 4, 0, 0], dtype=int32)

In [157]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_, allow_duplicates=True)

Downtown_merged = Downtown_Toronto_data

In [158]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

In [159]:
Downtown_merged = Downtown_merged.dropna() # clean up if there are any missing data

In [160]:
# create map with the clustered Neighborhoods
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighbourhood'], Downtown_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Finally, let's take a look at defining features in each cluster

In [161]:
#Cluster 0 appears to be dominated by a combination of coffee shops, bars, and restaurants
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,0.0,Coffee Shop,Bakery,Café,Mexican Restaurant,Distribution Center,Beer Store,Ice Cream Shop,French Restaurant,Chocolate Shop,Bank
5,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Italian Restaurant,Cosmetics Shop,Diner,Clothing Store,Beer Bar,Bakery,Breakfast Spot
6,Downtown Toronto,0.0,Coffee Shop,Beer Bar,Café,Bakery,Farmers Market,Cheese Shop,Cocktail Bar,Hotel,Fish Market,Basketball Stadium
7,Downtown Toronto,0.0,Coffee Shop,Italian Restaurant,Japanese Restaurant,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Department Store,Café,Chinese Restaurant,Gym / Fitness Center
9,Downtown Toronto,0.0,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Brewery,Fried Chicken Joint,Bakery,Bar,History Museum
10,Downtown Toronto,0.0,Coffee Shop,Hotel,Café,Gastropub,Bar,Italian Restaurant,Japanese Restaurant,American Restaurant,Deli / Bodega,Bakery
11,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Gastropub,Italian Restaurant,Japanese Restaurant,Bar
15,Downtown Toronto,0.0,Coffee Shop,Café,Beer Bar,Hotel,Japanese Restaurant,Cosmetics Shop,Art Gallery,Breakfast Spot,Creperie,Gym
16,Downtown Toronto,0.0,Coffee Shop,Café,Gym,Gastropub,Japanese Restaurant,Hotel,Asian Restaurant,American Restaurant,Bar,Beer Bar


In [162]:
#Cluster 1 is alone with a combination of Cafes and Bookstore (this uniquely seems to identify the University area on the map)
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,1.0,Café,Bookstore,Italian Restaurant,Japanese Restaurant,Bar,Bakery,Bank,Flower Shop,Chinese Restaurant,Beer Store


In [163]:
#Cluster 2 is more inland, with more of a focus on grocery store
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2.0,Grocery Store,Café,Coffee Shop,Diner,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports,Gas Station,Chocolate Shop


In [164]:
#Cluster 3 is the location near Airport
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3.0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Coffee Shop,Boutique


In [165]:
#Lastly, cluster 4 appears to be more of a mix (Music venues, clothing stores, bakery, gay bar)
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4.0,Music Venue,Museum,Coffee Shop,Cocktail Bar,Clothing Store,Church,Chocolate Shop,Chinese Restaurant,Cheese Shop,Caribbean Restaurant
1,Downtown Toronto,4.0,Coffee Shop,Italian Restaurant,Bakery,Café,Grocery Store,Breakfast Spot,Diner,Gastropub,Butcher,Gourmet Shop
2,Downtown Toronto,4.0,Coffee Shop,Japanese Restaurant,Gay Bar,Café,Gastropub,Mediterranean Restaurant,Men's Store,Bubble Tea Shop,Hotel,Burger Joint
4,Downtown Toronto,4.0,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Café,Middle Eastern Restaurant,Bookstore,Italian Restaurant,Bubble Tea Shop,Fast Food Restaurant
8,Downtown Toronto,4.0,Coffee Shop,Café,Bar,Cosmetics Shop,Bookstore,Bakery,Hotel,Asian Restaurant,Gym,Lounge
13,Downtown Toronto,4.0,Bar,Café,Coffee Shop,Bakery,Dumpling Restaurant,Mexican Restaurant,Grocery Store,Chinese Restaurant,Farmers Market,Cocktail Bar


Perhaps, with a cleaner categorization of venues, the neighborhoods will be clustered differently. In particular, if we were to combine Coffee shops and cafe, which appeared to be two of the more popular venues.